## From here just looking at MM

In [1]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

NameError: name 'pd' is not defined

In [ ]:
import requests
import pandas as pd
import numpy as np

probe_cc = "MM"
resp = requests.get('https://api.ooni.io/api/v1/aggregation'
         f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2023-07-01&until=2024-07-01&axis_x=domain&axis_y=probe_asn')
j = resp.json()
df = pd.json_normalize(j, record_path='result')
df['probe_cc'] = probe_cc
df['id'] = df['probe_cc'] + '_' + df['domain']
df['anomaly_ratio'] = (df['anomaly_count'] + df['confirmed_count']) / (df['measurement_count'])

In [ ]:
quantiles = df.groupby(['domain', 'probe_cc'])['measurement_count'].quantile(0.4).reset_index()

In [ ]:
quantiles = df.groupby(['domain', 'probe_cc'])['measurement_count'].quantile(0.4).reset_index()

In [ ]:
quantiles.rename(columns={'measurement_count': 'msmt_count_p40'}, inplace=True)

In [ ]:
df = df.merge(quantiles, on=['domain', 'probe_cc'], how='left')

In [ ]:
df

In [ ]:
df.loc[
    :, 'mult_factor'
] = 0.3

In [ ]:
df.loc[
    (df['measurement_count'] >= df['msmt_count_p40']) & (df['measurement_count'] > 5), 'mult_factor'
] = 1.0

In [ ]:
df.loc[
   :, 'w_anomaly_ratio'
] = df['mult_factor'] * df['anomaly_ratio']

In [ ]:
df[
    (df['domain'] == 'ooni.github.io')
]

In [ ]:
df.sort_values(by='w_anomaly_ratio', ascending=False)

In [ ]:
df_summary = df.groupby(['id','domain','probe_cc'])['w_anomaly_ratio']
df_summary = df_summary.agg([
('max', np.max),
('min', np.min),
('median', np.median),
('average', np.mean),
('P90', lambda x: np.percentile(x, 90)),
('P10', lambda x: np.percentile(x, 10)),
('P95', lambda x: np.percentile(x, 95)),
('P5', lambda x: np.percentile(x, 5))])

df_summary

In [ ]:
df_summary.reset_index(inplace=True)

In [ ]:
df_summary[
    df_summary['domain'] == 'ooni.github.io'
]

In [ ]:
df_summary.sort_values(by='P90', ascending=False)

In [ ]:
df['probe_asn'] = df['probe_asn'].astype(str)
df_summary_2 = df.groupby(['domain', 'probe_cc']).agg(
    w_anomaly_ratio=('w_anomaly_ratio', lambda x: x.quantile(0.9)),
    probe_asn_P90=('probe_asn', lambda x: ','.join(x))
).reset_index()
df_summary_2.sort_values(by='w_anomaly_ratio',ascending=False)

In [ ]:
df_summary_2[
    df_summary_2['domain'] == 'ooni.github.io'
]

In [ ]:
len(df_summary_2)

In [ ]:
df['probe_asn'] = df['probe_asn'].astype(str)

df_summary_3 = df.groupby(['domain', 'probe_cc']).agg(
    P90=('w_anomaly_ratio', lambda x: np.percentile(x, 90)),  # Calculate P90
    list_probe_asn_P90=('probe_asn', lambda x: ','.join(x[x.index[df.loc[x.index, 'w_anomaly_ratio'] > 0.9]]) if any(df.loc[x.index, 'w_anomaly_ratio'] > 0.9) else ''),  # Create list_probe_asn_blocking
    
    P95=('w_anomaly_ratio', lambda x: np.percentile(x, 95)),  # Calculate P95
    list_probe_asn_P95=('probe_asn', lambda x: ','.join(x[x.index[df.loc[x.index, 'w_anomaly_ratio'] > 0.95]]) if any(df.loc[x.index, 'w_anomaly_ratio'] > 0.95) else '')  # Create list_probe_asn_blocking

).reset_index()

df_summary_3.sort_values(by='P90',ascending=False)

## Trying for all countries

In [27]:
import requests
import pandas as pd
import numpy as np


# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN'
]

def get_webconnectivity(probe_cc):
    resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2023-07-01&until=2024-07-01&axis_x=domain&axis_y=probe_asn')
    j = resp.json()
    df = pd.json_normalize(j, record_path='result')
    df['probe_cc'] = probe_cc
    df['id'] = df['probe_cc'] + '_' + df['domain']
    df['probe_asn'] = df['probe_asn'].astype(str)
    df['probe_asn'] = 'AS' + df['probe_asn']
    df['anomaly_ratio'] = (df['anomaly_count'] + df['confirmed_count']) / (df['measurement_count'])
    
    quantiles = df.groupby(['domain', 'probe_cc'])['measurement_count'].quantile(0.4).reset_index()
    quantiles.rename(columns={'measurement_count': 'msmt_count_p40'}, inplace=True)
    df = df.merge(quantiles, on=['domain', 'probe_cc'], how='left')
    df.loc[:, 'mult_factor'] = 0.3
    df.loc[(df['measurement_count'] >= df['msmt_count_p40']) & (df['measurement_count'] > 5), 'mult_factor'] = 1.0
    df.loc[:, 'w_anomaly_ratio'] = df['mult_factor'] * df['anomaly_ratio']
        
    df_summary = df.groupby(['id','domain', 'probe_cc']).agg(
    P90=('w_anomaly_ratio', lambda x: np.percentile(x, 90)),  # Calculate P90
    list_probe_asn_P90=('probe_asn', lambda x: ', '.join(x[x.index[df.loc[x.index, 'w_anomaly_ratio'] > 0.9]]) if any(df.loc[x.index, 'w_anomaly_ratio'] > 0.9) else ''),  # Create list_probe_asn_blocking
    
    P95=('w_anomaly_ratio', lambda x: np.percentile(x, 95)),  # Calculate P95
    list_probe_asn_P95=('probe_asn', lambda x: ', '.join(x[x.index[df.loc[x.index, 'w_anomaly_ratio'] > 0.95]]) if any(df.loc[x.index, 'w_anomaly_ratio'] > 0.95) else '')  # Create list_probe_asn_blocking
    ).reset_index()
    
    return df_summary

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_domains_asn_all = pd.concat(df_list)

In [28]:
df_domains_asn_all[
    df_domains_asn_all['id'] == 'MM_ooni.github.io'
]

,id,domain,probe_cc,P90,list_probe_asn_P90,P95,list_probe_asn_P95
900,MM_ooni.github.io,ooni.github.io,MM,0.86,"AS136255, AS137424",1.0,"AS136255, AS137424"


In [29]:
len(df_domains_asn_all)

31545

In [30]:
df_domains_asn_all

,id,domain,probe_cc,P90,list_probe_asn_P90,P95,list_probe_asn_P95
0,MM_1.0.0.1,1.0.0.1,MM,0.000000,,0.000000,
1,MM_1.0.0.3,1.0.0.3,MM,0.000000,,0.004082,
2,MM_1.1.1.1,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
3,MM_1.1.1.1.com,1.1.1.1.com,MM,0.000000,,0.000000,
4,MM_1.1.1.3,1.1.1.3,MM,0.000000,,0.090323,
...,...,...,...,...,...,...,...
2291,VN_zendesk.com,zendesk.com,VN,0.270000,,0.285000,
2292,VN_zenvpn.net,zenvpn.net,VN,0.000000,,0.000000,
2293,VN_zh.wikipedia.org,zh.wikipedia.org,VN,0.002907,,0.004360,
2294,VN_zoom.us,zoom.us,VN,0.009600,,0.016800,


## Listing all the domains with start and end measurement date

In [31]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN'
]
def get_webconnectivity(probe_cc):
    resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2023-07-01&until=2024-07-01&axis_x=domain&axis_y=measurement_start_day')
    j = resp.json()
    df = pd.json_normalize(j, record_path='result')
    df['probe_cc'] = probe_cc
    df['measurement_start_day'] = pd.to_datetime(df['measurement_start_day'])
    df_summary = df.groupby(['domain','probe_cc'], as_index=False).agg(
        ok_count = ('ok_count', 'sum'),
        anomaly_count = ('anomaly_count', 'sum'),
        confirmed_count = ('confirmed_count', 'sum'),
        failure_count = ('failure_count', 'sum'),
        measurement_count = ('measurement_count', 'sum'),
        measurement_first_day = ('measurement_start_day', 'min'), 
        measurement_latest_day = ('measurement_start_day', 'max') 
        )
    df_summary['id'] = df_summary['probe_cc'] + '_' + df_summary['domain']
    return df_summary

# Note: as_index=False is to combine the first 2 headers when doing agg

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_domains_date_all = pd.concat(df_list)

In [32]:
df_domains_date_all

,domain,probe_cc,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1
3,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com
4,1.1.1.3,MM,471,7,0,2,480,2023-07-02,2024-06-27,MM_1.1.1.3
...,...,...,...,...,...,...,...,...,...,...
2291,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com
2292,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net
2293,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org
2294,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us


## Listing domains with category code

In [33]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN'
]
def get_webconnectivity(probe_cc):
    resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2023-07-01&until=2024-07-01&axis_x=domain&axis_y=category_code')
    j = resp.json()
    df = pd.json_normalize(j, record_path='result')
    df['probe_cc'] = probe_cc
    df['id'] = df['probe_cc'] + '_' + df['domain']
    df_cat = df[['domain', 'probe_cc', 'category_code', 'id']]
    return df_cat

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
    
df_all_category = pd.concat(df_list)

## Combining all tables together with anomaly ratio, category code and start/end measurement date

In [34]:
df_domains_all = pd.merge(df_domains_date_all,df_all_category,on='id',how='left')
df_domains_all = pd.merge(df_domains_all,df_domains_asn_all,on='id')


In [35]:
df_domains_all

,domain_x,probe_cc_x,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id,domain_y,probe_cc_y,category_code,domain,probe_cc,P90,list_probe_asn_P90,P95,list_probe_asn_P95
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1,1.0.0.1,MM,HOST,1.0.0.1,MM,0.000000,,0.000000,
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3,1.0.0.3,MM,HOST,1.0.0.3,MM,0.000000,,0.004082,
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,1.1.1.1,MM,ANON,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
3,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,1.1.1.1,MM,HOST,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
4,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com,NaN,NaN,NaN,1.1.1.1.com,MM,0.000000,,0.000000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31740,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com,NaN,NaN,NaN,zendesk.com,VN,0.270000,,0.285000,
31741,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net,zenvpn.net,VN,ANON,zenvpn.net,VN,0.000000,,0.000000,
31742,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org,zh.wikipedia.org,VN,CULTR,zh.wikipedia.org,VN,0.002907,,0.004360,
31743,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us,zoom.us,VN,COMT,zoom.us,VN,0.009600,,0.016800,


In [36]:
df_domains_all.to_csv('domain_list_2024.csv')

In [37]:
len(df_domains_all)

31745

## Confirmed blockings (by OONI) with blocking dates

In [38]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN'
]
def get_webconnectivity(probe_cc):
    resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2023-07-01&until=2024-07-01&axis_x=domain&axis_y=measurement_start_day')
    j = resp.json()
    df = pd.json_normalize(j, record_path='result')
    df['probe_cc'] = probe_cc
    df['measurement_start_day'] = pd.to_datetime(df['measurement_start_day'])
    df['confirmed_count'] = pd.to_numeric(df['confirmed_count'])
    confirmed_date = df[df['confirmed_count'] > 0].groupby(['domain','probe_cc'], as_index=False).agg(
        blocking_first_day = ('measurement_start_day', 'min'), 
        blocking_latest_day = ('measurement_start_day', 'max')
    )
    confirmed_date['id'] = confirmed_date['probe_cc'] + '_' + confirmed_date['domain']
    return confirmed_date

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
confirmed_date_all = pd.concat(df_list)

## Confirmed blockings with probe_asn blocking

In [39]:
import requests
import pandas as pd
# Burma, Cambodia, Hong Kong (China), India, Indonesia, Malaysia, Philippines, Thailand, Vietnam
cc_list = [
    'MM',
    'KH',
    'HK',
    'IN',
    'ID',
    'MY',
    'PH',
    'TH',
    'VN'
]
def get_webconnectivity(probe_cc):
    resp = requests.get('https://api.ooni.io/api/v1/aggregation'
             f'?probe_cc={probe_cc}&test_name=web_connectivity&since=2023-07-01&until=2024-04-01&axis_x=domain&axis_y=probe_asn')
    j = resp.json()
    df = pd.json_normalize(j, record_path='result')
    df['probe_cc'] = probe_cc
    df['confirmed_count'] = df['confirmed_count'].astype(int)
    df['probe_asn'] = df['probe_asn'].astype(str)
    df['probe_asn'] = 'AS' + df['probe_asn']
    confirmed_asn = df[df['confirmed_count'] > 0][['domain','probe_asn', 'probe_cc']]
    confirmed_asn = confirmed_asn.pivot_table(index=['domain', 'probe_cc'], values='probe_asn',
                                                aggfunc=lambda x: ', '.join(set(x))).reset_index().rename(columns = {
                                                'probe_asn' : 'probe_asn_blocking'})
    confirmed_asn['id'] = confirmed_asn['probe_cc'] + '_' + confirmed_asn['domain']
    return confirmed_asn

df_list = []
for cc in cc_list:
    df_list.append(pd.DataFrame(get_webconnectivity(cc)))
confirmed_asn_all = pd.concat(df_list)

In [40]:
confirmed_all = confirmed_date_all.merge(confirmed_asn_all, on='id', how='inner')
confirmed_all = confirmed_all[['domain_x', 
                        'probe_cc_x', 
                        'blocking_first_day',
                        'blocking_latest_day',
                        'probe_asn_blocking',
                        'id']].rename(columns = {
                        'domain_x' : 'domain',
                        'probe_cc_x' : 'probe_cc'
                        })

In [41]:
confirmed_all['is_confirmed_ooni'] = "TRUE"

In [42]:
confirmed_all

,domain,probe_cc,blocking_first_day,blocking_latest_day,probe_asn_blocking,id,is_confirmed_ooni
0,1221cd.org,MM,2023-12-28,2024-03-08,AS133385,MM_1221cd.org,TRUE
1,7day.news,MM,2023-12-28,2024-03-08,AS133385,MM_7day.news,TRUE
2,abs.twimg.com,MM,2023-07-09,2024-03-08,AS133385,MM_abs.twimg.com,TRUE
3,anonymouse.org,MM,2023-12-28,2024-03-08,AS133385,MM_anonymouse.org,TRUE
4,aungsanu.com,MM,2023-12-28,2024-03-05,AS133385,MM_aungsanu.com,TRUE
...,...,...,...,...,...,...,...
1461,www.xaluan.com,VN,2023-07-02,2024-06-25,AS7552,VN_www.xaluan.com,TRUE
1462,www.xvideos.com,VN,2024-01-24,2024-05-04,AS7552,VN_www.xvideos.com,TRUE
1463,www.youporn.com,VN,2024-01-24,2024-04-24,AS7552,VN_www.youporn.com,TRUE
1464,xinloiong.jonathanlondon.net,VN,2023-07-10,2024-05-05,AS7552,VN_xinloiong.jonathanlondon.net,TRUE


In [43]:
confirmed_all.to_csv('confirmed_ooni_Jun24.csv')

just copy everything from 'confirmed_ooni_Jun24.csv' to the Google Sheet. code below doesn't work!!!

## Confirmed by heuristics

In [44]:
confirmed_heuristics = pd.read_csv('confirmed_heuristics_Jun24.csv')

In [45]:
confirmed_heuristics

,input,probe_cc,blocking_method,blocking_count,probe_asn_blocking,id,is_confirmed_heuristics,check_duplicate,domain,period
0,https://hanhtinhtitanic.org/,VN,dns.confirmed,130,"AS45903, AS45899",VN_hanhtinhtitanic.org,TRUE,2,hanhtinhtitanic.org,Jul23-Apr24
1,https://kenhgamez.com/,VN,dns.confirmed,67,AS7552,VN_kenhgamez.com,TRUE,2,kenhgamez.com,Jul23-Apr24
2,https://www.giaitriluke.com/,VN,dns.confirmed,49,AS7552,VN_www.giaitriluke.com,TRUE,4,www.giaitriluke.com,Jul23-Apr24
3,https://www.haaretz.com/,VN,dns.confirmed,2,AS131429,VN_www.haaretz.com,TRUE,8,www.haaretz.com,Jul23-Apr24
4,https://www.haaretz.com/life/five-films-you-ha...,VN,dns.confirmed,4,AS131429,VN_www.haaretz.com,TRUE,8,www.haaretz.com,Jul23-Apr24
...,...,...,...,...,...,...,...,...,...,...
2761,https://www.lottovip.com/,TH,dns.confirmed,5,AS23969,TH_www.lottovip.com,TRUE,2,www.lottovip.com,Apr-June-2024
2762,https://www.newmandala.org/,TH,dns.confirmed,7,AS23969,TH_www.newmandala.org,TRUE,2,www.newmandala.org,Apr-June-2024
2763,https://www.no112.org/,TH,dns.confirmed,36,"AS24378, AS45758, AS17552, AS133481, AS45629, ...",TH_www.no112.org,TRUE,4,www.no112.org,Apr-June-2024
2764,https://www.pornhub.com/,TH,dns.confirmed,10,"AS45758, AS17552, AS23969",TH_www.pornhub.com,TRUE,5,www.pornhub.com,Apr-June-2024


In [46]:
df_domains_all

,domain_x,probe_cc_x,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id,domain_y,probe_cc_y,category_code,domain,probe_cc,P90,list_probe_asn_P90,P95,list_probe_asn_P95
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1,1.0.0.1,MM,HOST,1.0.0.1,MM,0.000000,,0.000000,
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3,1.0.0.3,MM,HOST,1.0.0.3,MM,0.000000,,0.004082,
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,1.1.1.1,MM,ANON,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
3,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,1.1.1.1,MM,HOST,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
4,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com,NaN,NaN,NaN,1.1.1.1.com,MM,0.000000,,0.000000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31740,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com,NaN,NaN,NaN,zendesk.com,VN,0.270000,,0.285000,
31741,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net,zenvpn.net,VN,ANON,zenvpn.net,VN,0.000000,,0.000000,
31742,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org,zh.wikipedia.org,VN,CULTR,zh.wikipedia.org,VN,0.002907,,0.004360,
31743,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us,zoom.us,VN,COMT,zoom.us,VN,0.009600,,0.016800,


In [47]:
df_domains_all.rename(columns={'domain_x': 'domain',
                                'probe_cc_x' : 'probe_cc',
                              },inplace=True)
df_domains_all

,domain,probe_cc,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id,domain_y,probe_cc_y,category_code,domain,probe_cc,P90,list_probe_asn_P90,P95,list_probe_asn_P95
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1,1.0.0.1,MM,HOST,1.0.0.1,MM,0.000000,,0.000000,
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3,1.0.0.3,MM,HOST,1.0.0.3,MM,0.000000,,0.004082,
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,1.1.1.1,MM,ANON,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
3,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,1.1.1.1,MM,HOST,1.1.1.1,MM,0.277660,"AS58952, AS212238",0.763926,"AS58952, AS212238"
4,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com,NaN,NaN,NaN,1.1.1.1.com,MM,0.000000,,0.000000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31740,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com,NaN,NaN,NaN,zendesk.com,VN,0.270000,,0.285000,
31741,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net,zenvpn.net,VN,ANON,zenvpn.net,VN,0.000000,,0.000000,
31742,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org,zh.wikipedia.org,VN,CULTR,zh.wikipedia.org,VN,0.002907,,0.004360,
31743,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us,zoom.us,VN,COMT,zoom.us,VN,0.009600,,0.016800,


In [48]:
df_domains_all = df_domains_all.merge(confirmed_all, on='id', how='left')
df_domains_all = df_domains_all.merge(confirmed_heuristics, on='id', how='left')

df_domains_all

,domain_x,probe_cc_x,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id,...,is_confirmed_ooni,input,probe_cc,blocking_method,blocking_count,probe_asn_blocking_y,is_confirmed_heuristics,check_duplicate,domain,period
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32821,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32822,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32823,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32824,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_domains_all

,domain_x,probe_cc_x,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id,...,is_confirmed_ooni,input,probe_cc,blocking_method,blocking_count,probe_asn_blocking_y,is_confirmed_heuristics,check_duplicate,domain,period
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32821,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32822,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32823,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32824,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_domains_all.rename(columns={'domain_x' : 'domain',
                               'probe_cc_x' : 'probe_cc',
                               'probe_asn_blocking_x' : 'probe_asn_blocking_ooni',
                               'probe_asn_blocking_y': 'probe_asn_blocking_heuristics'},inplace=True)

df_domains_all

,domain,probe_cc,ok_count,anomaly_count,confirmed_count,failure_count,measurement_count,measurement_first_day,measurement_latest_day,id,...,is_confirmed_ooni,input,probe_cc,blocking_method,blocking_count,probe_asn_blocking_heuristics,is_confirmed_heuristics,check_duplicate,domain,period
0,1.0.0.1,MM,346,0,0,2,348,2023-07-02,2024-06-27,MM_1.0.0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0.0.3,MM,387,1,0,2,390,2023-07-02,2024-06-27,MM_1.0.0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.1.1.1,MM,671,91,0,8,770,2023-07-02,2024-06-30,MM_1.1.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.1.1.1.com,MM,15,0,0,0,15,2024-06-03,2024-06-27,MM_1.1.1.1.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32821,zendesk.com,VN,3,3,0,0,6,2024-02-03,2024-02-03,VN_zendesk.com,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32822,zenvpn.net,VN,1524,0,0,3,1527,2024-01-05,2024-06-30,VN_zenvpn.net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32823,zh.wikipedia.org,VN,1583,1,0,0,1584,2023-07-01,2024-06-30,VN_zh.wikipedia.org,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32824,zoom.us,VN,1753,2,0,4,1759,2023-07-02,2024-06-30,VN_zoom.us,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_domains_all.to_csv('domain_list_2024.csv')

In [52]:
df_domains_all.drop('probe_asn_unique',inplace=True,axis=1)

KeyError: "['probe_asn_unique'] not found in axis"

In [ ]:
df_domains_all['probe_asn_unique'] = pd.DataFrame(pd.concat([df_domains_all['probe_asn_blocking_heuristics'], 
                        df_domains_all['probe_asn_blocking_ooni'], 
                        df_domains_all['list_probe_asn_P90']]).unique())

df_domains_all

In [ ]:
# Concatenate all ASN columns into one series and find unique values
asn_series = pd.concat([df['probe_asn_blocking_heuristics'], 
                        df['probe_asn_blocking_ooni'], 
                        df['probe_asn_P90']]).unique()

# Create a new dataframe with 'probe_asn_unique' column
df_unique_asn = pd.DataFrame({'probe_asn_unique': asn_series})

# Count the occurrences of each ASN in the original columns
df_asn_count = pd.DataFrame({
    'ASN': asn_series,
    'probe_asn_unique_count': [
        df['probe_asn_blocking_heuristics'].eq(asn_series).sum() +
        df['probe_asn_blocking_ooni'].eq(asn_series).sum() +
        df['probe_asn_P90'].eq(asn_series).sum()
        for asn_series in asn_series
    ]
})

# Merge df_unique_asn with df_asn_count to get the final dataframe
final_df = pd.merge(df_unique_asn, df_asn_count, left_on='probe_asn_unique', right_on='ASN')

In [ ]:
asn_series = pd.concat([df_domains_all['probe_asn_blocking_heuristics'], 
                        df_domains_all['probe_asn_blocking_ooni'], 
                        df_domains_all['list_probe_asn_P90']]).unique()

df_unique_asn = pd.DataFrame({'probe_asn_unique': asn_series})

df_unique_asn

In [ ]:
df_asn_count = pd.DataFrame({
    'ASN': asn_series,
    'probe_asn_unique_count': [
        df_domains_all['probe_asn_blocking_heuristics'].eq(asn_series).sum() +
        df_domains_all['probe_asn_blocking_ooni'].eq(asn_series).sum() +
        df_domains_all['list_probe_asn_P90'].eq(asn_series).sum()
        for asn_series in asn_series
    ]
})

df_asn_count

In [ ]:
# Merge df_unique_asn with df_asn_count to get the final dataframe
final_df = pd.merge(df_unique_asn, df_asn_count, left_on='probe_asn_unique', right_on='ASN')

# Merge final_df with df_domains_all based on 'probe_asn_unique'
final_df = pd.merge(df_domains_all, final_df, on=['probe_asn_blocking_heuristics', 'probe_asn_blocking_ooni', 'list_probe_asn_P90'], how='left')

final_df